## 1. 라이브러리 및 데이터 불러오기

In [302]:
import os
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup as bs
from tqdm.notebook import tqdm
from string import punctuation
import warnings
warnings.filterwarnings('ignore')

In [303]:
path = 'C:/Users/hanah/Desktop/workspace/Project/ESG/modelling/polarity_scoring/통계청/'
file_list = os.listdir(path)

In [304]:
df = pd.DataFrame()
for file in file_list:
    data = pd.read_excel(path + file)
    df = pd.concat([df, data])
    
df.reset_index(drop = True, inplace = True)

## 2. 라벨링 조정

In [305]:
df['라벨'] = df.iloc[:, 3:9].mean(axis=1)
df['라벨'][df['라벨'] <= 2.5] = 0
df['라벨'][df['라벨'] > 2.5] = 1
df['라벨'].value_counts()

1.0    4459
0.0    3159
Name: 라벨, dtype: int64

In [306]:
df = df[['제목', 'URL', '라벨']]
df

,제목,URL,라벨
0,"[신년사]손경식 CJ회장_ ""해외사업 공격적 확장_ 성장 가속화""",http://news.naver.com/main/read.nhn?mode=LSD&m...,1.0
1,[취업과 창업]신입사원 연차 생기고 공공기관 채용 역대 최대,http://news.naver.com/main/read.nhn?mode=LSD&m...,1.0
2,2018년 채용연계형 인턴채용 문 열렸다,http://news.naver.com/main/read.nhn?mode=LSD&m...,1.0
3,무술년 채용연계형 인턴채용 스타트,http://news.naver.com/main/read.nhn?mode=LSD&m...,1.0
4,그냥 쉬었다 청년_ 30만명 역대 최고치,http://news.naver.com/main/read.nhn?mode=LSD&m...,0.0
...,...,...,...
7613,코로나 충격에 따른 근로 시간 감소_ 과거 위기의 5배,https://news.naver.com/main/read.nhn?mode=LSD&...,0.0
7614,7월 사업체종사자 13만8천명 감소 코로나19 여파에 5개월째 감소,https://news.naver.com/main/read.nhn?mode=LSD&...,0.0
7615,한치 앞도 안보인다..채용문 꽁꽁 닫는 카드사,https://news.naver.com/main/read.nhn?mode=LSD&...,0.0
7616,토스_ 고객 상담직 신입 및 경력 30명 공개 채용,https://news.naver.com/main/read.nhn?mode=LSD&...,1.0


## 3. 기사 본문 크롤링 후 전처리

In [297]:
temp = pd.read_csv('newsbot.csv')
temp = temp.drop(['categories', 'dates'], axis=1)
temp.columns = ['기사', '제목']
temp

,기사,제목
0,[이데일리 이성기 기자] CJ그룹은 무술년(戊戌年) 새해를 맞아 국내사업에서의 압도...,"[신년사]손경식 CJ회장, ""해외사업 공격적 확장, 성장 가속화"""
1,[서울신문]청년 실업률이 치솟는 가운데 특별히 하는 일 없이 쉬는 청년이 지난해 3...,하는 일 없이 그냥 쉬는 청년들 30만명 넘었다
2,청년 실업률이 역대 최고 수준을 기록한 가운데 경제활동도 없이 쉬는 청년이 작년에 ...,"청년 30만명 ""그냥 쉬었다""...최악 취업난에 '니트족' 증가"
3,"계층 고착화로 청년 니트족(NEET: Not in education, employm...","일하기 포기한 청년 니트족, 역대 최고 수준 30만 돌파"
4,[한겨레]최저임금 인상이 청년고용에 미치는 부정적 영향은 없거나 통계적으로 무의미할...,ILO “최저임금 올라도 청년고용 영향 적어”
...,...,...
1033,국토교통부 산하 한국철도시설공단(철도공단)이 13년 가까이 용역업체에서 일한 정보통...,"철도공단, ‘예외 직종’ 내세워 전환에 난색"
1034,[아이뉴스24 채송무 기자] 김성태 자유한국당 원내대표가 정부의 최저임금 인상 속도...,"김성태 ""최저임금, 지금이라도 업종별 차등화 검토해야"""
1035,정부가 청년실업 해소를 위한 추가경정예산 편성을 검토하겠다고 밝히면서 그 규모에 관...,청년실업 해소 추경 규모는…20조 슈퍼추경 설도
1036,[이데일리 김정현 기자] 지난해 청년실업률은 9.9%. 지난 2000년 관련통계가 ...,[세모뉴스]경기 좋은 거 맞아? 청년실업은 왜 안 잡힐까요


In [298]:
temp = temp.dropna(axis=0)
temp = temp.loc[temp['기사'].str.len() > 100]
temp.drop_duplicates(subset=['기사'], inplace=True)

In [299]:
def clean_str(text):
    text = re.sub('[a-zA-Z-一-龥]','', text) # 영어, 숫자 제거
    text = re.sub('([ㄱ-ㅎㅏ-ㅣ]+)','', text)  # 자음, 모음 제거
    text = re.sub('<[^>]*>', '', text) # HTML 태그 제거 
    text = re.sub('(\【(.*?)\】)|(\[(.*?)\])|(\((.*?)\))', '', text) # 괄호안 글자 제거
    text = re.sub('[^\w\s^%^~^!^?^.]', ' ', text)  # 특수기호 제거
    pattern = '\.([^\.]*(?:기자|뉴스|특파원|교수|작가|대표|논설|주필|에세이|화백|사설|소장|단장|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|:앞쪽_화살표:|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)'
    text = re.sub(pattern, '', text) # 바이라인 제거    
    text = re.sub("\.+\.", '.', text)  # 다수의 점 (ex : ...) 점 한개로 대체
    text = re.sub(' +', ' ', text)   # 다수의 공백 축소
    text = re.sub('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', '', text) # 이메일 제거
    text = re.sub('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', text) # url 제거
    text = re.sub('([가-힣]{2,3}뉴스|[가-힣]{2,3} 기자)', '', text) # 매체, 기자 이름 제거
    text = text.replace('Copyrights', '').replace('무단전재 및 재배포 금지', '').replace('무단복제','').replace('무단배포','')
    text = text.replace('재배포 금지','').replace('무단 전재','')
    return text 

In [300]:
temp['기사'] = temp['기사'].map(clean_str)

In [311]:
temp.to_excel('temp_df.xlsx')

In [315]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7618 entries, 0 to 7617
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   제목      7618 non-null   object 
 1   URL     7606 non-null   object 
 2   라벨      7618 non-null   float64
dtypes: float64(1), object(2)
memory usage: 178.7+ KB


In [317]:
merge = pd.merge(df, temp, on='제목', how='left')

In [318]:
merge.to_excel('merge.xlsx')

In [ ]:
temp['제목'] = temp['제목'].map(clean_str)